# José Ligorría

## Regresión Lineal para predecir el valor de un inmueble, tipo casa

La idea es enconrar una expresión sencilla, lineal para describir el valor de una casa en terminos de una única variable y posiblemente una constante de sesgo en el modelo.
Por notación llamaremos $y$ al vecor que agrupa los valores del inmueble y a $x$ como la el vector que tiene los valores de la variable con la que se quiere inferir, la cual gracias a un estudio previo (https://github.com/Josealigo/Proyecto1_CDP/blob/master/Proyecto1-Jose_Ligorria.ipynb) se determina que esa variable es 'OverallQual'.
Entonces lo que se busca es lo siguiente:

$$\underset{\alpha,\beta} argmin \underset{x_{i} \in x}{\sum} \{ 0.5 * (f(x_{i},\alpha,\beta) - y_{i})^{2}) \}$$

Donde $$f(x_{i},\alpha,\beta) = \alpha * x_{i} + \beta$$

La metodología a seguir es crear el modelo lineal en código de python, considerando todo de forma vectorizada para hacer uso de las bondades del paquete tensorflow. Utilizando múltiples iteraciones a partir de un vector inicial de parámetros $\alpha$ y $\beta$ y corrigiendo para la siguiente iteración dicho vector con el vector gradiente del error, ya que sabemos que una función tiene sus óptimos (locales y globales) para cada punto de la función en dirección del vector gradiente, entonces ese vector de parémetros sería corregido de la siguiente forma: 

$$ (\alpha,\beta) = (\alpha,\beta) - lr * \nabla (\underset{x_{i} \in x}{\sum} \{ 0.5 * (f(x_{i},\alpha,\beta) - y_{i})^{2}) \} )$$

El código empleado para éste modelo es el siguiente:

In [1]:
import numpy as np
import tensorflow as tf
import math
import functools
from datetime import datetime
%load_ext tensorboard

In [2]:
if tf.__version__.startswith("2."):
    import tensorflow.compat.v1 as tf
    tf.compat.v1.disable_v2_behavior()
    tf.compat.v1.disable_eager_execution()
    print("Enabled compatitility to tf1.x")

Instructions for updating:
non-resource variables are not supported in the long term
Enabled compatitility to tf1.x


In [3]:
tf.reset_default_graph() 
def lazy_property(function):
    attribute = '_cache_' + function.__name__

    @property
    @functools.wraps(function)
    def decorator(self):
        if not hasattr(self, attribute):
            setattr(self, attribute, function(self))
        return getattr(self, attribute)

    return decorator

In [6]:
class Model:

    def __init__(self, data, target,learning_rate,data_size):
        self.data = data
        self.target = target
        self.learning_rate = learning_rate
        self.data_size = data_size
        self.m = tf.get_variable("parametros",dtype=tf.float32, shape=[2,1],
                    initializer=tf.zeros_initializer())
        self.prediction
        self.optimize
        self.error

    @lazy_property
    def prediction(self):
        tensor_piv = tf.concat([[self.data], [tf.constant(1.0,shape=[self.data_size])]], 0)
        pred_piv = tf.linalg.matmul(tensor_piv,self.m,transpose_a=True)
        tf.summary.histogram("prediccion", pred_piv)
        return pred_piv

    @lazy_property
    def optimize(self):
        gradiente = tf.gradients(self.error ,self.m )
        actualizacion_m = tf.assign(self.m, self.m - tf.math.multiply(self.learning_rate, gradiente[0]) )
        with tf.name_scope("Error"):
            error_piv = self.error
            error_summary = tf.summary.scalar(name = "ErrorSummary", tensor = error_piv)
        return actualizacion_m,error_summary
    
    

    @lazy_property
    def error(self):
        error_piv = 1/2*tf.reduce_mean(tf.math.square(self.target - self.prediction) )
        return error_piv


def main(learning_rate,epochs):
    data_inicial = np.load('proyecto_training_data.npy')
    np.random.seed(2)
    sub1 = np.random.choice(range(data_inicial.shape[0]), int(data_inicial.shape[0]*0.08))
    sub2 = np.setdiff1d(range(data_inicial.shape[0]),sub1)
    data_entrenamiento = data_inicial[sub1]
    data_validacion = data_inicial[sub2]
    data = tf.placeholder(tf.float32, [len(sub1)])
    target = tf.placeholder(tf.float32, [len(sub1)])
    model = Model(data, target,learning_rate,len(sub1))
    sess = tf.Session()
    sess.run(tf.initialize_all_variables())
    now = datetime.now()
    logdir = "./tf_logs/" + now.strftime("%Y%m%d-%H%M%S") +"_lr_"+str(learning_rate) + "_epochs_" + str(epochs)
    
    train_writer = tf.summary.FileWriter( logdir, sess.graph)

    for i in range(epochs):
        var_x, var_y = data_entrenamiento[:,1], data_entrenamiento[:,0]
        error_piv = sess.run(model.error, {data: var_x, target: var_y})
        if (i+1)%math.ceil(epochs/10) ==0:
            print("Error ", i+1, ": ",error_piv)
        optimizador = sess.run(model.optimize, {data: var_x, target: var_y})
        merge = tf.summary.merge_all()
        train_writer.add_summary(optimizador[1],i)
    train_writer.close()  

El modelo anterior genera el siguiente grafo en la metodología Tensorboard:
<img src="Imagenes/Grafo.png">

In [7]:
if __name__ == '__main__':
    tf.reset_default_graph() 
    main(0.1,300)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Error  30 :  inf
Error  60 :  inf
Error  90 :  nan
Error  120 :  nan
Error  150 :  nan
Error  180 :  nan
Error  210 :  nan
Error  240 :  nan
Error  270 :  nan
Error  300 :  nan


Experimento 1:
    Donde vemos que el learnig rate de 0.1 es ineficiente entonces hay que cambiarlo.

In [8]:
if __name__ == '__main__':
    tf.reset_default_graph() 
    main(0.01,300)

Error  30 :  2643587300.0
Error  60 :  2630591700.0
Error  90 :  2617889800.0
Error  120 :  2605475800.0
Error  150 :  2593343500.0
Error  180 :  2581484800.0
Error  210 :  2569896400.0
Error  240 :  2558569200.0
Error  270 :  2547499300.0
Error  300 :  2536680400.0


Experimento 2:
    Donde vemos que el learnig rate de 0.01 tiene resultados positivos.

In [9]:
if __name__ == '__main__':
    tf.reset_default_graph()
    main(0.001,300)

Error  30 :  4049424000.0
Error  60 :  2772107000.0
Error  90 :  2662503200.0
Error  120 :  2651984000.0
Error  150 :  2649875000.0
Error  180 :  2648480000.0
Error  210 :  2647153000.0
Error  240 :  2645829600.0
Error  270 :  2644511200.0
Error  300 :  2643196700.0


Experimento 3:
    Donde podemos ver que el learning rate de 0.01 tiene mejores resultados por lo que 0.001 entonces procedemos probando con 0.03.

In [10]:
if __name__ == '__main__':
    tf.reset_default_graph() 
    main(0.03,300)

Error  30 :  2618715000.0
Error  60 :  2582241500.0
Error  90 :  2548192800.0
Error  120 :  2516408000.0
Error  150 :  2486738200.0
Error  180 :  2459038500.0
Error  210 :  2433183000.0
Error  240 :  2409046300.0
Error  270 :  2386513700.0
Error  300 :  2365482500.0


Experimento 4:
    Donde vemos que el aprendizaje obtenido con la misma cantidad de Epochs es mejor que los anteriores.

In [11]:
if __name__ == '__main__':
    tf.reset_default_graph() 
    main(0.05,300)

Error  30 :  36567888000.0
Error  60 :  80951580000.0
Error  90 :  183474440000.0
Error  120 :  420180660000.0
Error  150 :  966586600000.0
Error  180 :  2227809000000.0
Error  210 :  5138880700000.0
Error  240 :  11858009000000.0
Error  270 :  27366570000000.0
Error  300 :  63162310000000.0


Experimento 5:
    Al tener un learning rate de 0.05 el error crece demasiado, por lo que probamos con 0.04.

In [12]:
if __name__ == '__main__':
    tf.reset_default_graph() 
    main(0.04,300)

Error  30 :  2606676500.0
Error  60 :  2559639800.0
Error  90 :  2516727300.0
Error  120 :  2477575700.0
Error  150 :  2441857000.0
Error  180 :  2409269800.0
Error  210 :  2379541000.0
Error  240 :  2352417500.0
Error  270 :  2327672000.0
Error  300 :  2305096400.0


Experimento 6:
    Vemos que para 0.04 tenemos hasta ahora la mejor optimización del error.

In [13]:
if __name__ == '__main__':
    tf.reset_default_graph() 
    main(0.045,300)

Error  30 :  2600848000.0
Error  60 :  2548713000.0
Error  90 :  2501774300.0
Error  120 :  2459440600.0
Error  150 :  2421259000.0
Error  180 :  2386820600.0
Error  210 :  2355764500.0
Error  240 :  2327750400.0
Error  270 :  2302487000.0
Error  300 :  2279699700.0


Experimento 7:
    Vemos que para 0.045 obtenemos una mejora en la optimización.

In [14]:
if __name__ == '__main__':
    tf.reset_default_graph() 
    main(0.049,300)

Error  30 :  5832955400.0
Error  60 :  3196544000.0
Error  90 :  2623283200.0
Error  120 :  2472503300.0
Error  150 :  2411075000.0
Error  180 :  2371045000.0
Error  210 :  2338286000.0
Error  240 :  2309620500.0
Error  270 :  2284129300.0
Error  300 :  2261373000.0


Experimento 8:
    Vemos que para 0.049 podemos mejorar un poco más la optimización. Por lo que ahora procedemos a ver si los dos mejores valores de learning rate pueden mejorar con más iteraciones.

In [15]:
if __name__ == '__main__':
    tf.reset_default_graph() 
    main(0.049,1000)

Error  100 :  2552929300.0
Error  200 :  2348673800.0
Error  300 :  2261373000.0
Error  400 :  2201616600.0
Error  500 :  2160538400.0
Error  600 :  2132297300.0
Error  700 :  2112884400.0
Error  800 :  2099536900.0
Error  900 :  2090361900.0
Error  1000 :  2084055300.0


In [16]:
if __name__ == '__main__':
    tf.reset_default_graph() 
    main(0.045,1000)

Error  100 :  2487177200.0
Error  200 :  2365762600.0
Error  300 :  2279699700.0
Error  400 :  2218695200.0
Error  500 :  2175454000.0
Error  600 :  2144802700.0
Error  700 :  2123073700.0
Error  800 :  2107673300.0
Error  900 :  2096756000.0
Error  1000 :  2089017900.0


Experimento 9:
    Vemos que para el learning rate de 0.049 se presentan los valores de error más pequeños. 

In [17]:
if __name__ == '__main__':
    tf.reset_default_graph() 
    main(0.049,6000)

Error  600 :  2132297300.0
Error  1200 :  2076736500.0
Error  1800 :  2070871600.0
Error  2400 :  2070253600.0
Error  3000 :  2070187000.0
Error  3600 :  2070179800.0
Error  4200 :  2070179800.0
Error  4800 :  2070179600.0
Error  5400 :  2070179000.0
Error  6000 :  2070178200.0


Experimento 10:
    Vemos que para el learning rate de 0.049 se presentan los valores de error más pequeños. 

Lo cual genera las siguiente gráfica de errores, apoyado en Tensorboard:
<img src="Imagenes/Errores.png">

Por otro lado, si suponemos la estimación para el valor de una casa como el promedio del valo de todas las casas tenemos el siguiente error:

In [28]:
data_inicial = np.load('proyecto_training_data.npy')
np.random.seed(2)
sub1 = np.random.choice(range(data_inicial.shape[0]), int(data_inicial.shape[0]*0.08))
sub2 = np.setdiff1d(range(data_inicial.shape[0]),sub1)
data_entrenamiento = data_inicial[sub1]
0.5*np.mean((data_entrenamiento[:,0]-np.mean(data_entrenamiento[:,0]))**2)

2070179531.6879828

### Conclusión

Dada la experimentación presentada se tiene que los mejores resultados los obtiene el learning rate de 0.049 y después de 6000 iteraciones se converge a un error de 2,070,178,200.0 en comparación con el estimado usando el promedio que es de 2,070,179,531.0 vemos que no es diferente, por lo que la estimación de ésta forma sujeto a los valores de entrenamiento no tiene un alto grado de acierto, menos se esperaría si se prueba con la información de prueba. Dadas esas consideraciones se tiene que la hipótesis de estimar el valor de la casa con la variable OverallQual en un modelo lineal no tiene un buen ajuste.